In [18]:
from bs4 import BeautifulSoup
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [19]:
import pandas as pd
import re
from multiprocessing import Pool
import pickle
import time

## Define elements for scraping

In [43]:
tabs = ['team_efficiency',
        'team_shooting_freq',
        'team_shooting_fg',
        'team_hc_and_putbacks',
        'team_transition',
       'opponent_shooting_freq',
       'opponent_shooting_fg',
       'opponent_hc_and_putbacks',
       'opponent_transition']
seasons = [str(season) for season in range(2004, 2022)]

my_headers = {
    'authority': 'cleaningtheglass.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="96", "Microsoft Edge";v="96"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.55 Safari/537.36 Edg/96.0.1054.43',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en,zh-CN;q=0.9,zh;q=0.8,en-US;q=0.7',
    'cookie': 'wordpress_logged_in_cfb45adfd4102bd74a046b78d76db012=zach225793%7C1670421579%7Cqe3pn3baHEn8lPIIQEpw2tAfbqsdUfINheMSscIPNl5%7Cce0105c6a9be760e3c27785cfabe16ec680f440f05609b1e800ffd1f561775b0; sessionid=yh4dn9hkv9ohcjzjge5twtl0g5skpi2m; csrftoken=DzYXKBmEics8LjNpMZl7B7szJOBuxQOMtO496G8DI2aSwt9pdwJ4TvUkrdAY8GMj',
}

In [47]:
def data_per_team(season):
    def add_id_season(table, season):
        
        ids = [re.search(r'\d+', str(html_soup.find('table').find_all('a')[i]))[0] for i in range(len(table))]
        table['id'] = ids
        table.columns = [' '.join(col).strip() for col in table.columns.values]
        table['season'] = season + "-" + str(int(season)+1)[2:]
        
        return table
    
    season_tables = []
    for team in range(1, 31):
        # start session and create response
        session = requests.Session()
        retry = Retry(connect=3, backoff_factor=0.5)
        adapter = HTTPAdapter(max_retries=retry)
        session.mount('http://', adapter)
        session.mount('https://', adapter)
        
        if team == 4 and season == '2003':
            session.keep_alive = False
            continue
        
        response = ""
        while response == "":
            try:
                url = "https://cleaningtheglass.com/stats/team/" + str(team) + "/onoff?season=" + season + "#tab-team_efficiency"
                response = session.get(url, headers=my_headers)
                response_status = re.search(r'\d+', str(response))[0]
                if response_status != '200': # handle fail case from response
                    break
            except ChunkedEncodingError as err:
                time.sleep(25)
                continue

        # parse
        html_soup = BeautifulSoup(response.text, 'html.parser')
        tables = [pd.read_html(str(html_soup.find_all('table')[i]))[0] for i in range(9)]
        team_tables = [add_id_season(table, season) for table in tables]

        session.keep_alive = False
        
        season_tables.append(team_tables)
    return season_tables


with Pool(12) as p:
    onoff_stats = p.map(data_per_team, seasons)

In [ ]:
with open('onoff_stats.pickle', 'wb') as handle:
    pickle.dump(onoff_stats, handle, protocol=pickle.HIGHEST_PROTOCOL)